In [4]:
!pip install sentence-transformers transformers streamlit scikit-learn pandas

In [5]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer
import pandas as pd
import numpy as np


In [6]:
# Load tokenizer only once
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def compute_complexity(prompt):
    tokens = tokenizer.tokenize(prompt)
    return len(tokens)


c:\Users\shaar\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaar\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
def estimate_energy(layers, training_time, flops_per_hour, complexity_score):
    flops_total = flops_per_hour * training_time
    kwh = 0.000001 * layers * flops_total * complexity_score  # toy formula
    return round(kwh, 3)


In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define a few optimized alternatives
ALTERNATIVES = [
    "Summarize this document.",
    "Explain this briefly.",
    "Write a short overview.",
    "Give key points only.",
    "Rephrase in simple terms."
]

def suggest_optimized_prompt(prompt):
    prompt_vec = model.encode(prompt, convert_to_tensor=True)
    alt_vecs = model.encode(ALTERNATIVES, convert_to_tensor=True)
    sims = util.cos_sim(prompt_vec, alt_vecs)
    best_idx = sims.argmax()
    return ALTERNATIVES[best_idx]


c:\Users\shaar\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaar\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

In [9]:
# 👤 User input manually
prompt_text = "Write a 300-word essay on the evolution of AI."
model_layers = 24
training_time = 2.5  # in hours
flops_per_hour = 12e9  # 12 billion FLOPs/hour


In [10]:
# 1. Compute complexity
complexity = compute_complexity(prompt_text)

# 2. Predict energy
energy_kwh = estimate_energy(model_layers, training_time, flops_per_hour, complexity)

# 3. Suggest simplified version
simplified = suggest_optimized_prompt(prompt_text)

# 4. Output
print("🔍 Original Prompt:", prompt_text)
print("📊 Token Complexity:", complexity)
print("⚡ Estimated Energy:", energy_kwh, "kWh")
print("💡 Suggested Simpler Prompt:", simplified)


🔍 Original Prompt: Write a 300-word essay on the evolution of AI.
📊 Token Complexity: 12
⚡ Estimated Energy: 8640000.0 kWh
💡 Suggested Simpler Prompt: Write a short overview.


c:\Users\shaar\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [11]:
import streamlit as st

st.title("Prompt Energy Estimator")

prompt = st.text_area("Enter your prompt here")
layers = st.number_input("Model Layers", value=12)
time = st.number_input("Training Time (hrs)", value=1.0)
flops = st.number_input("FLOPs per hour (e.g., 12e9)", value=12e9)

if st.button("Estimate Energy"):
    if prompt:
        complexity = compute_complexity(prompt)
        energy = estimate_energy(layers, time, flops, complexity)
        suggestion = suggest_optimized_prompt(prompt)
        st.markdown(f"**Complexity:** {complexity} tokens")
        st.markdown(f"**Estimated Energy:** {energy} kWh")
        st.markdown(f"**Suggested Simpler Prompt:** {suggestion}")


2025-07-25 17:56:35.191 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 17:56:35.925 
  command:

    streamlit run C:\Users\shaar\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-25 17:56:35.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 17:56:35.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 17:56:35.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 17:56:35.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 17:56:35.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 17:56:35.936 Thre